In [1]:
import numpy as np
import qutip as qu
import matplotlib.pyplot as plt


In [2]:
N = 2 #dimension of Hilbert Space
G = np.array([qu.identity(N), qu.sigmax(), qu.sigmay(), qu.sigmaz()])/np.sqrt(N)

mat_gg = np.zeros([N**2, N**2, N**2, N**2], dtype=np.complex64)
for i, gi in enumerate(list(G)):
    for j, gj in enumerate(list(G)):
        # print(i, j)
        mat_gg[i, j] = np.kron(np.conjugate(gj), gi)

In [3]:
H0 = np.array([[0.127, 0.25], [0.25, 0]])

gamma = -0.05
sigma_p = qu.sigmap().full()
sigma_m = qu.sigmam().full()

In [4]:
A = np.array(
    [-1j * H0 - 0.5 * gamma * sigma_m @ sigma_p, np.identity(2), gamma * sigma_p]
)
Bdag = np.array([np.identity(2), 1j * H0 - 0.5*gamma*  sigma_m @ sigma_p, sigma_m])

In [5]:
# I am gonna create the superoperator version
superop = 0
for i in range(3):
    # print(i)
    # print(np.kron(np.transpose(Bdag[i]), A[i]))
    superop = superop + np.kron(np.transpose(Bdag[i]), A[i])

In [6]:
superop

array([[ 0.   +0.j   ,  0.   -0.25j ,  0.   +0.25j , -0.05 +0.j   ],
       [ 0.   -0.25j ,  0.025+0.127j,  0.   +0.j   ,  0.   +0.25j ],
       [ 0.   +0.25j ,  0.   +0.j   ,  0.025-0.127j,  0.   -0.25j ],
       [ 0.   +0.j   ,  0.   +0.25j ,  0.   -0.25j ,  0.05 +0.j   ]])

In [7]:
def make_liouvillian(H):
    return -1j * np.kron(H.T, np.identity(2)) +1j * np.kron(np.identity(2), H)

def spre(A): 
    return np.kron(np.eye(2), A)

def spost(B):
    return np.kron(B.T, np.eye(2))

def sprepost(A,B):
    return np.kron(B.T, A)


In [8]:
-1j*spre(H0) + 1j*spost(H0) + gamma*sprepost(sigma_p, sigma_m)  - 0.5*gamma*spre(sigma_m@sigma_p) - 0.5*gamma*spost(sigma_m@sigma_p) 

array([[ 0.   +0.j   ,  0.   -0.25j ,  0.   +0.25j , -0.05 +0.j   ],
       [ 0.   -0.25j ,  0.025+0.127j,  0.   +0.j   ,  0.   +0.25j ],
       [ 0.   +0.25j ,  0.   +0.j   ,  0.025-0.127j,  0.   -0.25j ],
       [ 0.   +0.j   ,  0.   +0.25j ,  0.   -0.25j ,  0.05 +0.j   ]])

In [9]:
# Now, I should be able to compute the chi_ij matrix

In [10]:
chi = np.zeros([4, 4], dtype=np.complex64)

for i in range(4):
    for j in range(4):
        for k in range(3):
            chi[i, j]= chi[i, j] + np.trace(G[i]@A[k])*np.trace(G[j]@Bdag[k])

In [11]:
def qu_L(L):
    return qu.Qobj(np.array(L), dims=[[[2], [2]], [[2], [2]]], shape=(4, 4))

In [12]:
qu_L(superop)


Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.   +0.j     0.   -0.25j   0.   +0.25j  -0.05 +0.j   ]
 [ 0.   -0.25j   0.025+0.127j  0.   +0.j     0.   +0.25j ]
 [ 0.   +0.25j   0.   +0.j     0.025-0.127j  0.   -0.25j ]
 [ 0.   +0.j     0.   +0.25j   0.   -0.25j   0.05 +0.j   ]]

In [13]:

qu.to_super(qu.to_chi(qu_L(superop)))

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.   +0.j     0.   -0.25j   0.   +0.25j  -0.05 +0.j   ]
 [ 0.   -0.25j   0.025+0.127j  0.   +0.j     0.   +0.25j ]
 [ 0.   +0.25j   0.   +0.j     0.025-0.127j  0.   -0.25j ]
 [ 0.   +0.j     0.   +0.25j   0.   -0.25j   0.05 +0.j   ]]

In [14]:
print(np.round(chi, 3))

[[ 0.05 -0.j     0.   +0.5j    0.   +0.j    -0.025+0.127j]
 [ 0.   -0.5j   -0.025+0.j     0.   +0.025j  0.   +0.j   ]
 [ 0.   +0.j     0.   -0.025j -0.025+0.j     0.   +0.j   ]
 [-0.025-0.127j  0.   +0.j     0.   +0.j     0.   +0.j   ]]


In [24]:
example_dij = qu.rand_herm(3, seed=8)
example_dij

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[ 0.        +0.j          0.49939635-0.2055391j   0.29966336+0.11729425j]
 [ 0.49939635+0.2055391j  -0.53454334+0.j         -0.58624984+0.33060707j]
 [ 0.29966336-0.11729425j -0.58624984-0.33060707j  0.04534934+0.j        ]]

In [27]:
make_liouvillian(H0)

array([[0.+0.j   , 0.+0.25j , 0.-0.25j , 0.+0.j   ],
       [0.+0.25j , 0.-0.127j, 0.+0.j   , 0.-0.25j ],
       [0.-0.25j , 0.+0.j   , 0.+0.127j, 0.+0.25j ],
       [0.+0.j   , 0.-0.25j , 0.+0.25j , 0.+0.j   ]])

In [29]:
superop_sample_dij = np.einsum("ijkl, ij -> kl", mat_gg[1:, 1:], example_dij)
superop_sample_dij = superop_sample_dij + make_liouvillian(H0)

In [34]:
qu.to_chi(qu_L(make_liouvillian(H0)))/2


Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True, superrep = chi
Qobj data =
[[0.+0.j    0.-0.5j   0.+0.j    0.-0.127j]
 [0.+0.5j   0.+0.j    0.+0.j    0.+0.j   ]
 [0.+0.j    0.+0.j    0.+0.j    0.+0.j   ]
 [0.+0.127j 0.+0.j    0.+0.j    0.+0.j   ]]

In [36]:
qu_L(superop_sample_dij)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.02267467+0.j          0.31513521+0.60177204j  0.31513521-0.60177204j
  -0.06173257+0.j        ]
 [-0.01547185+0.01552221j -0.02267467-0.127j       0.26727167+0.49939635j
  -0.31513521+0.10177204j]
 [-0.01547185-0.01552221j  0.26727167-0.49939635j -0.02267467+0.127j
  -0.31513521-0.10177204j]
 [-0.47281077+0.j          0.01547185-0.48447779j  0.01547185+0.48447779j
   0.02267467+0.j        ]]

In [38]:
qu.to_super(qu.to_chi(qu_L(superop_sample_dij))/2)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.01133734+0.j          0.15756761+0.30088602j  0.15756761-0.30088602j
  -0.03086629+0.j        ]
 [-0.00773593+0.0077611j  -0.01133734-0.0635j      0.13363584+0.24969818j
  -0.15756761+0.05088602j]
 [-0.00773593-0.0077611j   0.13363584-0.24969818j -0.01133734+0.0635j
  -0.15756761-0.05088602j]
 [-0.23640539+0.j          0.00773593-0.2422389j   0.00773593+0.2422389j
   0.01133734+0.j        ]]